<a href="https://colab.research.google.com/github/jessicangu/nmt-research-project/blob/main/final_viet_eng_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

initializing project and checkpoints onto google drive. cloning openNMT-py.

In [ ]:
from google.colab import drive
import os
import random
from collections import Counter
import sentencepiece as spm

drive.mount('/content/drive', force_remount = True)

!mkdir -p /content/drive/MyDrive/final_viet_eng_nmt/
!mkdir -p /content/drive/MyDrive/final_viet_eng_nmt/checkpoints
!mkdir -p /content/drive/MyDrive/final_viet_eng_nmt/data
!mkdir -p /content/drive/MyDrive/final_viet_eng_nmt/data/huggingface
!mkdir -p /content/drive/MyDrive/final_viet_eng_nmt/processed
!mkdir -p /content/drive/MyDrive/final_viet_eng_nmt/models
!mkdir -p /content/drive/MyDrive/final_viet_eng_nmt/vocab

def step_completed(checkpoint_file):
    """checking if a processing step has already been completed."""
    return os.path.exists(f'/content/drive/MyDrive/final_viet_eng_nmt/{checkpoint_file}')

def mark_step_completed(checkpoint_file):
    """mark a processing step as completed."""

    checkpoint_dir = os.path.dirname(f'/content/drive/MyDrive/final_viet_eng_nmt/{checkpoint_file}')
    if checkpoint_dir:
        !mkdir -p {checkpoint_dir}
    !touch /content/drive/MyDrive/final_viet_eng_nmt/{checkpoint_file}

if not step_completed('checkpoints/dependencies_installed'):
    print("installing dependencies...")
    !pip install OpenNMT-py scikit-learn sacrebleu sacremoses sentencepiece

    if not os.path.exists('/content/OpenNMT-py'):
        print("cloning OpenNMT-py...")
        !git clone https://github.com/OpenNMT/OpenNMT-py /content/OpenNMT-py
    else:
        print("OpenNMT-py directory already exists. skipping clone.")

    mark_step_completed('checkpoints/dependencies_installed')
else:
    print("dependencies already installed.")

    if not os.path.exists('/content/OpenNMT-py'):
        !git clone https://github.com/OpenNMT/OpenNMT-py

Mounted at /content/drive
dependencies already installed.


installing dependencies/packages

In [ ]:
!pip install --upgrade pip setuptools wheel

!apt-get -y update
!apt-get install -y libcairo2-dev

!pip install --no-cache-dir OpenNMT-py scikit-learn sacrebleu mosestokenizer sacremoses sentencepiece torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 numpy==1.23.5 jedi pycairo datasets protobuf==3.20.1 transformers evaluate bert-score

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 257 kB in 1s (223 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package list

establishing directories for huggingface as well as setting up datasets for 70% training, 10% validation, and 20% testing

In [ ]:
!mkdir -p /content/drive/MyDrive/final_viet_eng_nmt/data/medev_200k

In [ ]:
!mkdir -p /content/medev_raw
!mkdir -p /content/medev_200k

!wget https://huggingface.co/datasets/nhuvo/MedEV/resolve/main/train.en.txt -O /content/medev_raw/train.en.txt
!wget https://huggingface.co/datasets/nhuvo/MedEV/resolve/main/train.vi.txt -O /content/medev_raw/train.vi.txt

!wget https://huggingface.co/datasets/nhuvo/MedEV/resolve/main/val.en.new.txt -O /content/medev_raw/val.en.txt
!wget https://huggingface.co/datasets/nhuvo/MedEV/resolve/main/val.vi.new.txt -O /content/medev_raw/val.vi.txt

!wget https://huggingface.co/datasets/nhuvo/MedEV/resolve/main/test.en.new.txt -O /content/medev_raw/test.en.txt
!wget https://huggingface.co/datasets/nhuvo/MedEV/resolve/main/test.vi.new.txt -O /content/medev_raw/test.vi.txt


import random
from mosestokenizer import MosesTokenizer

def load_parallel_tokenized(en_path, vi_path):
    with open(en_path, "r", encoding="utf-8") as f_en, open(vi_path, "r", encoding="utf-8") as f_vi:
        en_lines = f_en.read().splitlines()
        vi_lines = f_vi.read().splitlines()
    assert len(en_lines) == len(vi_lines)
    with MosesTokenizer('en') as en_tok, MosesTokenizer('vi') as vi_tok:
        return [(" ".join(en_tok(en)), " ".join(vi_tok(vi))) for en, vi in zip(en_lines, vi_lines)]

train_pairs = load_parallel_tokenized("/content/medev_raw/train.en.txt", "/content/medev_raw/train.vi.txt")
val_pairs = load_parallel_tokenized("/content/medev_raw/val.en.txt", "/content/medev_raw/val.vi.txt")
test_pairs = load_parallel_tokenized("/content/medev_raw/test.en.txt", "/content/medev_raw/test.vi.txt")

all_pairs = train_pairs + val_pairs + test_pairs
random.shuffle(all_pairs)
subset_200k = all_pairs[:200000]

train_70 = subset_200k[:140000]
val_10 = subset_200k[140000:160000]
test_20 = subset_200k[160000:]

def write_subset(pairs, out_prefix):
    with open(f"/content/medev_200k/{out_prefix}.en", "w", encoding="utf-8") as f_en, \
         open(f"/content/medev_200k/{out_prefix}.vi", "w", encoding="utf-8") as f_vi:
        for en_line, vi_line in pairs:
            f_en.write(en_line.strip() + "\n")
            f_vi.write(vi_line.strip() + "\n")

write_subset(train_70, "train")
write_subset(val_10, "val")
write_subset(test_20, "test")

!cp /content/medev_200k/* /content/drive/MyDrive/final_viet_eng_nmt/data/medev_200k

--2025-05-09 12:28:18--  https://huggingface.co/datasets/nhuvo/MedEV/resolve/main/train.en.txt
Resolving huggingface.co (huggingface.co)... 18.160.143.99, 18.160.143.76, 18.160.143.75, ...
Connecting to huggingface.co (huggingface.co)|18.160.143.99|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/fa/df/fadf989523804cdd6598c4071e561d70a6e942ac9b685a454c874750368c363b/c2ef096c662ffe7bef21ed21fa87061403d95ed041a7e2b00af41099523748e7?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27train.en.txt%3B+filename%3D%22train.en.txt%22%3B&response-content-type=text%2Fplain&Expires=1746797298&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0Njc5NzI5OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2ZhL2RmL2ZhZGY5ODk1MjM4MDRjZGQ2NTk4YzQwNzFlNTYxZDcwYTZlOTQyYWM5YjY4NWE0NTRjODc0NzUwMzY4YzM2M2IvYzJlZjA5NmM2NjJmZmU3YmVmMjFlZDIxZmE4NzA2MTQwM2Q5NWVkMDQxYTdlMmIwMGFmNDEwOTk1MjM

checking and validating my datasets before I proceed

In [ ]:
,def check_file(file_path, num_lines=5):
    """reads in a file and prints the total number of lines and the first few lines."""
    print(f"checking: {file_path}")
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            lines = f.read().splitlines()
        total_lines = len(lines)
        print("total lines:", total_lines)
        print("first {} lines:".format(num_lines))
        for i, line in enumerate(lines[:num_lines]):
            print(f"{i}: {line.strip()}")

        empty_count = sum(1 for line in lines if not line.strip())
        if empty_count:
            print(f"warning: Found {empty_count} empty lines.")
        print("-" * 40)
    except Exception as e:
        print(f"error reading {file_path}: {e}")

files_to_check = [
    "/content/drive/MyDrive/final_viet_eng_nmt/data/medev_200k/train.en",
    "/content/drive/MyDrive/final_viet_eng_nmt/data/medev_200k/train.vi",
    "/content/drive/MyDrive/final_viet_eng_nmt/data/medev_200k/val.en",
    "/content/drive/MyDrive/final_viet_eng_nmt/data/medev_200k/val.vi",
    "/content/drive/MyDrive/final_viet_eng_nmt/data/medev_200k/test.en",
    "/content/drive/MyDrive/final_viet_eng_nmt/data/medev_200k/test.vi"
]

for file_path in files_to_check:
    check_file(file_path)


checking: /content/drive/MyDrive/final_viet_eng_nmt/data/medev_200k/train.en
total lines: 140000
first 5 lines:
0: Conclusion : There is the significant statistically relationship between Hepatitis C virus infection and type 2 Diabetes Mellitus disease with RR = 3.9 ( 95 % CI = 2.16 - 7.33 ) . Furthermore , Hepatitis C virus infection has the strongest correlation with type 2 Diabetes Mellitus disease ( RR = 5.86 , 95 % CI = 2.79 - 12.28 ) versus other risk factors such as age , overweight condition and history of women having child with birthweight equal to or greater than 4 Kg
1: Carbapenem is realistic selection , but resistance rates were high for A baumannii and P aeruginosa .
2: Most of these reactions were minor and did not require any specific treatment .
3: The rate of appendix rupture diagnosed by CT accounted was 4.3 % .
4: Conclusion : study on 4059 pupils aging from 6 - 10 showed that the prevalence of overweight @-@ obesity was 17.5 % including 10.8 % of overweight and 6.

double checking directories

In [ ]:
!chmod -R 777 /content/drive/MyDrive/final_viet_eng_nmt/models
!chmod -R 777 /content/drive/MyDrive/final_viet_eng_nmt/data
!chmod -R 777 /content/drive/MyDrive/final_viet_eng_nmt 2>/dev/null

!ls -la /content/OpenNMT-py

import os
print("models directory exists:", os.path.exists("/content/drive/MyDrive/final_viet_eng_nmt/models"))
print("data directory exists:", os.path.exists("/content/drive/MyDrive/final_viet_eng_nmt/data"))
print("OpenNMT-py directory exists:", os.path.exists("/content/OpenNMT-py"))

!cd /content/OpenNMT-py && python -c "import onmt; print('OpenNMT-py version:', onmt.__version__)"

import numpy as np
if not hasattr(np, 'dtypes'):
    np.dtypes = np.sctypeDict

total 128
drwxr-xr-x 12 root root  4096 May  9 08:26 .
drwxr-xr-x  1 root root  4096 May  9 12:25 ..
drwxr-xr-x  2 root root  4096 May  9 08:26 available_models
-rw-r--r--  1 root root   100 May  9 08:26 build_vocab.py
-rw-r--r--  1 root root 18237 May  9 08:26 CHANGELOG.md
drwxr-xr-x  2 root root  4096 May  9 08:26 config
-rw-r--r--  1 root root  4126 May  9 08:26 CONTRIBUTING.md
drwxr-xr-x  5 root root  4096 May  9 08:26 data
drwxr-xr-x  2 root root  4096 May  9 08:26 docker
drwxr-xr-x  3 root root  4096 May  9 08:26 docs
drwxr-xr-x  5 root root  4096 May  9 08:26 eval_llm
drwxr-xr-x  8 root root  4096 May  9 08:26 .git
drwxr-xr-x  3 root root  4096 May  9 08:26 .github
-rw-r--r--  1 root root  1265 May  9 08:26 .gitignore
-rw-r--r--  1 root root  1072 May  9 08:26 LICENSE.md
drwxr-xr-x 14 root root  4096 May  9 08:35 onmt
-rw-r--r--  1 root root  8761 May  9 08:26 README.md
-rw-r--r--  1 root root   119 May  9 08:26 requirements.opt.txt
-rw-r--r--  1 root root   313 May  9 08:26 SEC

loading in transformer models, set up training arguments

In [ ]:
from transformers import MBart50TokenizerFast, MBartForConditionalGeneration, EarlyStoppingCallback, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from datasets import load_dataset, Dataset, DatasetDict

model_name = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

def load_parallel_dataset(src_file, tgt_file, src_lang, tgt_lang):
    with open(src_file, 'r', encoding='utf-8') as f_src, open(tgt_file, 'r', encoding='utf-8') as f_tgt:
        src_lines = f_src.read().strip().split('\n')
        tgt_lines = f_tgt.read().strip().split('\n')
    assert len(src_lines) == len(tgt_lines)
    return Dataset.from_dict({src_lang: src_lines, tgt_lang: tgt_lines})

inbound_datasets = DatasetDict({
    "train": load_parallel_dataset("/content/medev_200k/train.vi", "/content/medev_200k/train.en", "vi", "en"),
    "validation": load_parallel_dataset("/content/medev_200k/val.vi", "/content/medev_200k/val.en", "vi", "en"),
    "test": load_parallel_dataset("/content/medev_200k/test.vi", "/content/medev_200k/test.en", "vi", "en"),
})


tokenizer.src_lang = "vi_VN"
tokenizer.tgt_lang = "en_XX"

def preprocess(batch):
    return tokenizer(
        batch["vi"],
        text_target=batch["en"],
        max_length=256,
        padding="max_length",
        truncation=True,
    )

def is_valid(example):
    return (
        example["vi"] is not None and isinstance(example["vi"], str) and example["vi"].strip() != "" and
        example["en"] is not None and isinstance(example["en"], str) and example["en"].strip() != ""
    )

for split in ["train", "validation", "test"]:
    inbound_datasets[split] = inbound_datasets[split].filter(is_valid)

tokenized_inbound = inbound_datasets.map(
    preprocess,
    batched=True,
    remove_columns=["vi", "en"]
)

model.config.forced_bos_token_id = tokenizer.lang_code_to_id["en_XX"]

training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/mbart_vi_en_200k",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=5,
    predict_with_generate=True,
    generation_num_beams=6,
    label_smoothing_factor=0.1,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    gradient_accumulation_steps=2,
    logging_steps=100,
    report_to="none",
    fp16=True,
    load_best_model_at_end=True,
    remove_unused_columns=False
)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_inbound["train"],
    eval_dataset=tokenized_inbound["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

Filter:   0%|          | 0/140000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/20000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/140000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

<ipython-input-33-b8049f942780>:76: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


training

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.726400,1.724398
2,1.697800,1.712342
3,1.673100,1.708228
4,1.649700,1.711294


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_bos_token_id': 250004}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=35000, training_loss=1.77824261605399, metrics={'train_runtime': 12707.2265, 'train_samples_per_second': 55.087, 'train_steps_per_second': 3.443, 'total_flos': 3.0339816751104e+17, 'train_loss': 1.77824261605399, 'epoch': 4.0})

evaluate scores

In [ ]:
import evaluate
import numpy as np
from sacremoses import MosesDetokenizer

def evaluate_direction(tokenized_dataset, raw_dataset, src_lang_code, tgt_lang_code, tgt_field):
    tokenizer.src_lang = src_lang_code
    model.config.forced_bos_token_id = tokenizer.lang_code_to_id[tgt_lang_code]

    predictions = trainer.predict(tokenized_dataset)
    decoded_preds = tokenizer.batch_decode(predictions.predictions, skip_special_tokens=True)
    actual_refs = [r for r in raw_dataset[tgt_field]]

    decoded_preds = [pred.strip().lower() for pred in decoded_preds]
    actual_refs = [ref.strip().lower() for ref in actual_refs]

    detok = MosesDetokenizer(lang="en")
    decoded_preds = [detok.detokenize(pred.split()) for pred in decoded_preds]
    actual_refs = [[detok.detokenize(ref.split())] for ref in actual_refs]

    bleu = evaluate.load("sacrebleu")
    meteor = evaluate.load("meteor")
    ter = evaluate.load("ter")
    bertscore = evaluate.load("bertscore")

    print(f"\n--- {src_lang_code} → {tgt_lang_code} ---")
    print("BLEU:", bleu.compute(predictions=decoded_preds, references=actual_refs)["score"])
    print("METEOR:", meteor.compute(predictions=decoded_preds, references=actual_refs)["meteor"])
    print("TER:", ter.compute(predictions=decoded_preds, references=actual_refs)["score"])
    print("BERTScore (F1):", np.mean(bertscore.compute(predictions=decoded_preds, references=[r[0] for r in actual_refs], lang='en')["f1"]))


In [ ]:
evaluate_direction(
    tokenized_dataset=tokenized_inbound["test"],
    raw_dataset=inbound_datasets["test"],
    src_lang_code="vi_VN",
    tgt_lang_code="en_XX",
    tgt_field="en"
)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...



--- vi_VN → en_XX ---
BLEU: 41.228674555559536
METEOR: 0.6684550153252498
TER: 52.331777418919486


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore (F1): 0.9443479986026884


vi_VN → en_XX

BLEU: 41.228674555559536

METEOR: 0.6684550153252498

TER: 52.331777418919486

BERTScore: 0.9443479986026884

saving model in drive

In [ ]:
trainer.save_model("/content/drive/MyDrive/final_viet_eng_nmt/models/mbart_vien_200k_final")
tokenizer.save_pretrained("/content/drive/MyDrive/final_viet_eng_nmt/models/mbart_vien_200k_final")

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'forced_bos_token_id': 250024}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


('/content/drive/MyDrive/final_viet_eng_nmt/models/mbart_vien_200k_final/tokenizer_config.json',
 '/content/drive/MyDrive/final_viet_eng_nmt/models/mbart_vien_200k_final/special_tokens_map.json',
 '/content/drive/MyDrive/final_viet_eng_nmt/models/mbart_vien_200k_final/sentencepiece.bpe.model',
 '/content/drive/MyDrive/final_viet_eng_nmt/models/mbart_vien_200k_final/added_tokens.json',
 '/content/drive/MyDrive/final_viet_eng_nmt/models/mbart_vien_200k_final/tokenizer.json')

In [ ]:
model = MBartForConditionalGeneration.from_pretrained(model_name)

outbound_datasets = DatasetDict({
    "train": load_parallel_dataset("/content/medev_200k/train.en", "/content/medev_200k/train.vi", "en", "vi"),
    "validation": load_parallel_dataset("/content/medev_200k/val.en", "/content/medev_200k/val.vi", "en", "vi"),
    "test": load_parallel_dataset("/content/medev_200k/test.en", "/content/medev_200k/test.vi", "en", "vi"),
})
for split in ["train", "validation", "test"]:
    outbound_datasets[split] = outbound_datasets[split].filter(is_valid)

tokenizer.src_lang = "en_XX"
tokenizer.tgt_lang = "vi_VN"
tokenized_outbound = outbound_datasets.map(preprocess, batched=True, remove_columns=["en", "vi"])

training_args_outbound = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/mbart_en_vi_200k",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=5,
    predict_with_generate=True,
    generation_num_beams=6,
    label_smoothing_factor=0.1,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    gradient_accumulation_steps=2,
    logging_steps=100,
    report_to="none",
    fp16=True,
    load_best_model_at_end=True,
    remove_unused_columns=False
)

trainer_outbound = Seq2SeqTrainer(
    model=model,
    args=training_args_outbound,
    train_dataset=tokenized_outbound["train"],
    eval_dataset=tokenized_outbound["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

trainer_outbound.train()


Filter:   0%|          | 0/140000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/20000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/140000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

<ipython-input-42-3b6a1df4dd62>:38: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer_outbound = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss
1,1.726800,1.724403
2,1.697800,1.712348
3,1.673100,1.708045
4,1.649600,1.710946


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=35000, training_loss=1.7782031773158482, metrics={'train_runtime': 13151.0922, 'train_samples_per_second': 53.228, 'train_steps_per_second': 3.327, 'total_flos': 3.0339816751104e+17, 'train_loss': 1.7782031773158482, 'epoch': 4.0})

In [ ]:
evaluate_direction(tokenized_outbound["test"], outbound_datasets["test"], "en_XX", "vi_VN", "vi")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!



--- en_XX → vi_VN ---
BLEU: 41.30111163997908
METEOR: 0.5620678744071113
TER: 52.62700921496285


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore (F1): 0.8808819587484003


en_XX → vi_VN

BLEU: 41.30111163997908

METEOR: 0.5620678744071113

TER: 52.62700921496285

BERTScore: 0.8808819587484003

In [ ]:
model.save_pretrained("/content/drive/MyDrive/mbart_vi_en_200k/model")
tokenizer.save_pretrained("/content/drive/MyDrive/mbart_vi_en_200k/tokenizer")

model.save_pretrained("/content/drive/MyDrive/mbart_en_vi_200k/model")
tokenizer.save_pretrained("/content/drive/MyDrive/mbart_en_vi_200k/tokenizer")


('/content/drive/MyDrive/mbart_en_vi_200k/tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/mbart_en_vi_200k/tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/mbart_en_vi_200k/tokenizer/sentencepiece.bpe.model',
 '/content/drive/MyDrive/mbart_en_vi_200k/tokenizer/added_tokens.json',
 '/content/drive/MyDrive/mbart_en_vi_200k/tokenizer/tokenizer.json')